In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from KaggleWord2VecUtility import KaggleWord2VecUtility

C:\Dev\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# ****** Define functions to create average word vectors *
# Why average? Since the IMDB dataset is the variable-length reviews,
# so we need to transform individual word vectors into a feature set with
# the same length. The method is to 'average' the word vectors.

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    # 
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype='float32')
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
            
    #
    # Devide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Pre-allocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    #
    # Loop through the reviews
    for review in reviews:
        #
        # Print a status message every 1000th review
        if counter%1000. == 0.:
            print('Review %d of %d' % (counter, len(reviews)))
        #
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
                                                        num_features)
        #
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews['review']:
        clean_reviews.append(KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True))
    return clean_reviews           

In [3]:
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('testData.tsv', header=0, delimiter='\t', quoting=3)
unlabeled_train = pd.read_csv('unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

# Verify the number of reviews that were read (100, 000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
     "and %d unlabeled reviews\n" % (train["review"].size,
     test["review"].size, unlabeled_train["review"].size ) )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [4]:
# Load the punkt tokenizer
# punkt helps sentence splitting
# word2vec needs setence lists of splitted words for its input
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = [] # Initialize an empty list of sentences

In [6]:
print('Parsing sentences from training set')
for review in train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    
print('Parsing sentences from unlabeled set')
for review in unlabeled_train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    # .append can't be used when appending a list of lists to another list of lists

Parsing sentences from training set


C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:198: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beaut

Parsing sentences from unlabeled set


C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:207: UserWarning: "b'http://www.archive.org/details/LovefromaStranger"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:207: UserWarning: "b'http://www.loosechangeguide.com/LooseChangeGuide.html"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
C:\Dev\Anaconda3\

In [8]:
# ****** Set parameters and train the word2vec model
#
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
                   level=logging.INFO)

# Set values for various parameters
num_features = 300  # Word vecor dimensionality
min_word_count = 40 # Minimum word count
                    ### Any word that doesn't occur at least this
                    ### across all documents is ignored
num_workers = 4     # Number of threads to run in parallel
context = 10        # Context window size
downsampling = 1e-3 # Downsample setting for frequent words
# *** Default parameters
# Architecture: skip-gram (tends to slower than cBOW, but better performs)
# Training algorithm: Hierachical softmax (over negative sampling)


# Initialize and train the model (this will take some time)
print("Training word2vec model...")
model = Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample=downsampling, seed=1)

# If you doon't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training word2vec model...


In [9]:
model.doesnt_match("man woman child kitchen".split())

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':


'kitchen'

In [10]:
model.doesnt_match("france england germany berlin".split())

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':


'berlin'

In [11]:
model.doesnt_match("paris berlin london austria".split())

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':


'austria'

In [12]:
model.most_similar("man")

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('woman', 0.6282485723495483),
 ('lady', 0.592788577079773),
 ('lad', 0.5731804370880127),
 ('millionaire', 0.5404776334762573),
 ('monk', 0.5335522294044495),
 ('soldier', 0.5268305540084839),
 ('farmer', 0.5250458717346191),
 ('men', 0.521050214767456),
 ('guy', 0.5185298323631287),
 ('chap', 0.5064795017242432)]

In [13]:
model.most_similar("queen")

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('princess', 0.6782366633415222),
 ('bride', 0.6036782264709473),
 ('latifah', 0.6011467576026917),
 ('stepmother', 0.5982118844985962),
 ('maid', 0.5954171419143677),
 ('brunette', 0.5926809310913086),
 ('belle', 0.5852283239364624),
 ('vamp', 0.5832668542861938),
 ('victoria', 0.5805691480636597),
 ('elizabeth', 0.5778067111968994)]

In [14]:
model.most_similar("awful")

C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('terrible', 0.7715333700180054),
 ('atrocious', 0.7461475729942322),
 ('horrible', 0.7415310740470886),
 ('abysmal', 0.7074114680290222),
 ('dreadful', 0.7035708427429199),
 ('horrendous', 0.6947226524353027),
 ('horrid', 0.6788488626480103),
 ('appalling', 0.6769682168960571),
 ('lousy', 0.6434138417243958),
 ('amateurish', 0.620899498462677)]

In [15]:
# ****** Create average vectors for the training and test sets
#
print("Creating average feature vecs for training reviews")

trainDataVecs = getAvgFeatureVecs( getCleanReviews(train), model, num_features )

print("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( getCleanReviews(test), model, num_features )


# ****** Fit a random forest to the training set, then make predictions
#
# Fit a random forest to the training data, using 100 trees
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results
result = forest.predict( testDataVecs )

# Write the test results
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3 )
print("Wrote Word2Vec_AverageVectors.csv")

C:\Dev\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Creating average feature vecs for training reviews
Review 0 of 25000


C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [16]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1
